In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from dataloader import JPXData
import helper_functions as hf
import matplotlib.pyplot as plt
from autoencoder import autoencoder as ae

In [ ]:
stock_prices = pd.read_csv("modified_data/autoencoder_data.csv")

In [ ]:
# ~ 3 month testing time frame 2022-01-04 to 2022-04-28
train_data = stock_prices[stock_prices["Date"] < '2022-01-04'].copy().reset_index(drop=True)
test_data  = stock_prices[stock_prices["Date"] >= '2022-01-04'].copy().reset_index(drop=True)

In [ ]:
train_class = JPXData(data=train_data, model="ae")
trainloader = torch.utils.data.DataLoader(train_class, batch_size=1, shuffle=True, num_workers=5)
test_class  = JPXData(data=test_data, model="ae")
testloader  = torch.utils.data.DataLoader(test_class, batch_size=1, shuffle=True, num_workers=5)

In [ ]:
net = ae(e_layers=[69, 65, 60, 55, 50, 45, 40, 35, 30, 25, 20, 15], d_layers=[15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 69]).cuda()

In [ ]:
criterion = nn.MSELoss().cuda()
optimizer = optim.SGD(net.parameters(), lr=1e-4, momentum=0.9)

In [ ]:
loss_a1 = []

for epoch in range(400):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels, date = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = hf.loss_function(outputs, labels, criterion=nn.L1Loss().cuda())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    
        if i % 1000 == 999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 1000:.10f}')
            loss_a1.append(running_loss/1000)
            running_loss = 0.0
        # torch.save(net.state_dict(), f'saved_models/{epoch}.pt')

print('Finished Training')

In [ ]:
final_loss = np.array(loss_a1)
plt.plot(final_loss)
plt.show()